In [ ]:
## Creamos la funciones auxiliares

In [ ]:
import os.path
import subprocess

In [ ]:
BIN_DIR = '../../build/'
BENCH_DIR = '../../bench/'

DEFAULT_BENCH_TIME = 0.1

`getCheckpoint()` nos indica cuantas mediciones ya hicimos, y tenemeos que saltear

In [ ]:
def getCheckpoint(file):
    if not os.path.isfile(file):
        return 0
    with open(file) as f:
        lines = sum(1 for line in f)
    return lines

`execute` es un wrapper sobre `subprocess` para que se printee stdout mientras se va ejecutando

In [ ]:
def execute(args):
    popen = subprocess.Popen(args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, universal_newlines=True)
    stdout_lines = iter(popen.stdout.readline, "")
    for stdout_line in stdout_lines:
        yield stdout_line
        
    popen.stdout.close()
    return_code = popen.wait()
    if return_code != 0:
        raise subprocess.CalledProcessError(return_code, args)

`doBench()` corre los benchmarks necesarios, sin repetir los que ya se hicieron

In [ ]:
def doBench(algoritmo, out, variables, extraArgs='', minTime=None, reps=4, casos=1, quiet=False, checkpointOffset=0):
    executable = BIN_DIR + algoritmo
    output = BENCH_DIR + algoritmo + '-' + out
    minTime = minTime if minTime is not None else DEFAULT_BENCH_TIME
    
    checkpoint = max(0, getCheckpoint(output) - checkpointOffset)
    if checkpoint > 0 and quiet < 2:
        print("Salteando", checkpoint, "combinaciones")
    
    args = executable
    args += " -o " + output
    args += " -a " + variables + " " + str(reps) + " " + str(casos)
    args += " -c " + str(checkpoint)
    args += " " + extraArgs
    if minTime > 0:
        args += " -A " + str(minTime)

    args = args.split()
    for line in execute(args):
        if quiet:
            print('.', end='')
        else:
            print(line, end='')
    
    if quiet < 2:
        print('Done!')

## Benchmarks de tiempos para cada algoritmo

### En funcion de la cantidad de nodos, mochila de tamaño $\infty$

#### Exacto bruteforce (todas las permutaciones)

In [ ]:
doBench('exacto', 'time-bruteforce', '2:5 2:5 1000', '-p none', casos=5, quiet=True)

In [ ]:
doBench('exacto', 'timemoch-bruteforce', '5 5 4:20', '-p none', casos=5, quiet=True)

#### Exacto bruteforce (podado)

In [ ]:
doBench('exacto', 'time-backtracking', '2:5 2:5 1000', '-p backtracking', casos=5, quiet=True)

In [ ]:
doBench('exacto', 'timemoch-backtracking', '5 5 4:20', '-p backtracking', casos=5, quiet=True)

#### Greedy (closest first)

In [ ]:
doBench('greedy', 'time-closest', '2:5 2:5 1000',quiet=True)
doBench('greedy', 'time-closest-extended',
        '1000:10000:1000 1000:10000:1000 100000000', quiet=True)

In [ ]:
doBench('greedy', 'timemoch-closest', '2500 2500 100:7500:200', casos=1, quiet=True)

#### Greedy (furthest first)

In [ ]:
doBench('greedy', 'time-furthest', '2:5 2:5 1000', '-t furthest', quiet=True)
doBench('greedy', 'time-furthest-extended',
        '1000:10000:1000 1000:10000:1000 100000000',
        '-t furthest', quiet=True)

In [ ]:
doBench('greedy', 'timemoch-furthest', '2500 2500 100:7500:200', '-t furthest', casos=1, quiet=True)

In [ ]:
doBench('exacto', 'timemoch-bruteforce', '5 5 4:20', '-p none', casos=5, quiet=True)

#### Local (2 opt)

In [ ]:
doBench('local', 'time-2opt', '2:5 2:5 1000', '-t dos_opt', quiet=True)
doBench('local', 'time-2opt-extended', '10:100:10 10:100:10 100000', '-t dos_opt', quiet=True)

In [ ]:
doBench('local', 'timemoch-2opt', '50 50 10:150', '-t dos_opt', casos=1, quiet=True)

#### Local (swap)

In [ ]:
doBench('local', 'time-swap', '2:5 2:5 1000', '-t swap', quiet=True)
doBench('local', 'time-swap-extended', '10:100:10 10:100:10 100000', '-t swap', quiet=True)

In [ ]:
doBench('local', 'timemoch-swap', '50 50 10:150', '-t swap', casos=1, quiet=True)

#### Local (swap_min)

In [ ]:
doBench('local', 'time-swap_min', '2:5 2:5 1000', '-t swap_min', quiet=True)
doBench('local', 'time-swap_min-extended', '10:100:10 10:100:10 100000', '-t swap_min', quiet=True)

#### Grasp

In [ ]:
doBench('grasp', 'time-grasp', '2:5 2:5 1000', quiet=True)
doBench('grasp', 'time-grasp-extended', '10:50:5 10:50:5 100000', quiet=True)

In [ ]:
doBench('grasp', 'timemoch-11', '20 20 10:60', casos=2, quiet=True)

## Benchmarks de precisión de las heurísticas

#### Comparando con el exacto

In [ ]:
for gen in ['random', 'separated', 'zigzag']:
    print(gen+"Generator")
    
    counter = -1
    skip = getCheckpoint(BENCH_DIR + 'exacto-result-small-' + gen)

    for gyms in range(1,12):
        for stops in range(1, 13-gyms):
            for mochila in (3, stops, stops*2, stops*3):
                counter += 1
                seed = gyms * 5 + stops * 3 + mochila * 2
                if counter < skip:
                    continue

                variables = "{} {} {}".format(gyms, stops, mochila)
                f = " -s {} {} -G {} ".format(seed,seed,gen)
                doBench('exacto', 'result-small-'+gen,
                        variables, f, minTime=0, checkpointOffset=counter, quiet=2)
                doBench('greedy', 'result-small-'+gen+'-closest',
                        variables, f + '-t closest', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('greedy', 'result-small-'+gen+'-furthest',
                        variables, f + '-t furthest', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('local', 'result-small-'+gen+'-2opt',
                        variables, f + '-t dos_opt', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('local', 'result-small-'+gen+'-swap',
                        variables, f + '-t swap', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('grasp', 'result-small-'+gen,
                        variables, f, minTime=0, checkpointOffset=counter, quiet=2)

print("Done!")

#### Comparando con el greedy

In [ ]:
for gen in ['random', 'separated', 'zigzag']:
    print(gen+"Generator")
    
    counter = -1
    skip = getCheckpoint(BENCH_DIR + 'greedy-result-big-'+gen+'-closest')

    for gyms in range(10,30):
        for stops in range(10, 40-gyms):
            for mochila in (3, stops/2, stops, stops*2, stops*3):
                counter += 1
                seed = gyms * 5 + stops * 3 + mochila * 2
                if counter < skip:
                    continue

                variables = "{} {} {}".format(gyms, stops, mochila)
                f = " -s {} {} -G {} ".format(seed,seed,gen)
                doBench('greedy', 'result-big-'+gen+'-closest',
                        variables, f + '-t closest', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('greedy', 'result-big-'+gen+'-furthest',
                        variables, f + '-t furthest', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('local', 'result-big-'+gen+'-2opt',
                        variables, f + '-t dos_opt', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('local', 'result-big-'+gen+'-swap',
                        variables, f + '-t swap', minTime=0, checkpointOffset=counter, quiet=2)
                doBench('grasp', 'result-big-'+gen,
                        variables, f, minTime=0, checkpointOffset=counter, quiet=2)
            
print("Done!")

## Benchmarks para las variables de GRASP

### Tiempo

In [ ]:
skip = getCheckpoint(BENCH_DIR + 'grasp-variables')
seeds = [123,789,654]
counter = -len(seeds)

for i in range(1,20):
    inicios = i/10
    for l in range(1,20):
        limite = l/10
        
        counter += len(seeds)
        if counter < skip:
            continue

        c = len(seeds)
        for seed in seeds:
            c -= 1
            f = ' -L {} -I {} -s {} {} '.format(limite,inicios,seed,seed)
            doBench('grasp', 'variables', '25 25 100000', f,
                    checkpointOffset=counter-c, quiet=2)

print("Done!")